In [1]:
import torch
import torch.nn as nn

In [2]:
class ConvNet(nn.Module):
    """
    Define the convolutional neural network.
    """

    def __init__(self, input_channels, output_channels, kernel_size, stride, padding):
        super(ConvNet, self).__init__()
        """
        Set the number of input and output channels, kernel size, stride, and padding.
        """

        self.input_channels = input_channels
        self.output_channels = output_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

    def forward(self, x):
        """
        Get the batch size and dimensions of the input tensor.
        """

        batch_size, _, input_width, input_height = x.size()

        # Compute the output width and height.
        #
        output_width = (input_width + 2 * self.padding - self.kernel_size) // self.stride + 1
        output_height = (input_height + 2 * self.padding - self.kernel_size) // self.stride + 1

        # Create output tensor with the specified dimensions.
        #
        output = torch.zeros(batch_size, self.output_channels, output_width, output_height)

        # Loop through the batch and apply the convolution operation to each sample.
        #
        for i in range(batch_size):
            for j in range(self.output_channels):   # Loop through the output channels.
                for k in range(output_height):  # Loop through the output height.
                    for l in range(output_width):   # Loop through the output width.
                        output_pos = (i, j, k, l)   # Loop through the output width.

                        ## Compute the start and end indices for the kernel.
                        # along the width and height dimensions.
                        #
                        start_w = l * self.stride
                        end_w = start_w + self.kernel_size
                        start_h = k * self.stride
                        end_h = start_h + self.kernel_size

                        # Get the submatrix of the input tensor that the kernel will be applied to.
                        #
                        input_submatrix = x[i, :, start_w:end_w, start_h:end_h]

                        # Apply the convolution operation to the submatrix and store the result in the output tensor.
                        #
                        output[output_pos] = torch.sum(input_submatrix)

        return output

    def backward(self, output_grad):
        """
        Use PyTorch's Automatic Differentiation to compute the gradients of the output 
        w.r.t. the model's parameters and the input tensor.
        """

        output_grad.backward()

In [3]:
if __name__ == "__main__":
    """ 
    Create a convolutional neural network
    """

    model = ConvNet(input_channels=1, output_channels=1, kernel_size=3, stride=1, padding=1)

    # Define the input tensor.
    #
    x = torch.randn(1, 1, 5, 5)

    # Apply the convolutional neural network to the input tensor and print output.
    #
    output = model(x)
    print(output)

    # Move the model to the GPU
    #
    # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # model.to(device)

tensor([[[[ 2.5389,  2.3328,  1.0332, -0.2631, -1.7795],
          [ 2.6077,  3.4282, -0.4960, -1.6046, -2.6826],
          [-0.1072, -0.2893, -2.0724, -3.0228, -2.4961],
          [-0.8663,  0.0636, -2.7949, -3.3064, -2.4923],
          [-1.4686, -1.7661, -2.3913, -2.3756, -1.4972]]]])
